## 練習題 1

請啟用 playwright 瀏覽Hotels.com (https://tw.hotels.com/) 網站的資料，查詢12/11~12/12 礁溪寒沐酒店是否有兩位大人的空房間，並將playwright 產生的程式碼寫入hotels.py 之中

## 練習題 2
請修改以下程式碼，讓我們能夠瀏覽到底 12月 有哪個週末我們可以訂到礁溪老爺酒店的房間?

In [ ]:
from playwright.sync_api import Playwright, sync_playwright
from datetime import date, datetime, timedelta


page_url = "https://tw.hotels.com/ho620495360/?q-check-in={}&q-check-out={}&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=BEST_SELLER"

def run(playwright: Playwright) -> None:
    browser = playwright.webkit.launch(headless=False)
    context = browser.new_context()

    # Open new page
    page = context.new_page()

    sdate = date(2021,11, 6)
    edate = date(2021,11, 7)
    for i in range(0,4):
        sdt = sdate + timedelta(days = i * 7)
        edt = edate + timedelta(days = i * 7)
        page.goto(page_url.format(sdt, edt))
        page.screenshot(path = '{}_{}.png'.format(sdt, edt))
    
    # ---------------------
    context.close()
    browser.close()


with sync_playwright() as playwright:
    run(playwright)


## 練習題 3
請修改以下程式碼，讓我們能夠瀏覽到底 12月 各週末[礁溪老爺酒店」的房間個要花多少錢?

In [ ]:
from playwright.sync_api import Playwright, sync_playwright
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

page_url = "https://tw.hotels.com/search.do?destination-id=1639064&q-check-in={}&q-check-out={}&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&f-name={}&sort-order=BEST_SELLER"

def getWeekend(start_date, end_date):
    sdate = datetime.strptime(start_date, '%Y-%m-%d')
    edate = datetime.strptime(end_date, '%Y-%m-%d')
    delta = edate - sdate
    weekends = []
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        if (day.strftime('%w') == '6'):
            weekends.append((day.strftime('%Y-%m-%d'), (day + timedelta(days=1)).strftime('%Y-%m-%d')) )
    return weekends

def run(playwright: Playwright) -> None:
    browser = playwright.webkit.launch(headless=False)
    context = browser.new_context()
    # Open new page
    page = context.new_page()
    weekends = getWeekend('2021-11-01', '2021-12-01')
    for sdate, edate in weekends:
        page.goto(page_url.format(sdate,edate,'礁溪寒沐酒店, 礁溪, 台灣'))
        time.sleep(3)
        soup = BeautifulSoup(page.content(), 'lxml')
        price = soup.select_one('li[data-hotel-id] span[dir="auto"]')
        if price:
            print(sdate,edate,price.text)

    context.close()
    browser.close()
with sync_playwright() as playwright:
    run(playwright)